In [31]:
import sys
import os
import pandas as pd
from pathlib import Path

grid_search = pd.DataFrame()

RES_PATH = Path(f"../res/")
for f in os.listdir(RES_PATH):
    if "gridSearch" not in f:
        continue
    df = pd.read_pickle(RES_PATH / f)
    grid_search = pd.concat([grid_search, df], ignore_index=True)

In [33]:
grid_search.T

,0,1,2,3,4,5,6
epoch,2,2,2,2,2,2,2
dur,9.43103,11.0436,10.2441,34.9944,39.5594,39.2247,90.1806
train_loss,0.200445,0.150642,0.183069,0.103082,0.0971755,0.0987672,0.0945318
valid_loss,0.145836,0.129438,0.13752,0.0883356,0.0817219,0.0881664,0.0823609
valid_accuracy_score,0.57688,0.563412,0.547699,0.666564,0.654545,0.552542,0.665331
valid_f1_score,0.711994,0.709918,0.706909,0.593233,0.651755,0.700742,0.600735
valid_recall_score,0.958848,0.979424,1,0.456862,0.607412,0.984366,0.473075
valid_precision_score,0.566221,0.556725,0.546682,0.845659,0.703083,0.544,0.822759
valid_roc_auc_score,0.538683,0.521811,0.502469,0.681,0.65779,0.522815,0.678566
hyperparam_batch_size,1024,1024,1024,1024,1024,1024,1024
